# Library Import

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

import analyzer
import neural_network as nn
import convolutional_neural_network as cnn
import visualizer

# Training Data Set Preparation

In [11]:
resolution = [250,250]
# Resolution should be consistent throughout the file

In [ ]:

# Because the training dataset are individual, the list of each data just contain themselves.
data1 = analyzer.Data("MORE_DATA/db_Y_0027.okc", resolution) 
data2 = analyzer.Data("MORE_DATA/db_Y_0030.okc", resolution)


# Training The Model

In [13]:
# Run this section if you want to use NN model

array1, headers1, non_nan_indices1, num_grids1 = nn.data_arranger(data1.df)
array2, headers2, non_nan_indices2, num_grids2 = nn.data_arranger(data2.df)

# The learning rate, batch size, and epochs are proven to be working.

nn_model = nn.model_create_compile(headers1, 0.05)

nn_model, loss_hist = nn.model_train(nn_model, array1, 1000, 10)
nn_model, loss_hist = nn.model_train(nn_model, array2, 1000, 10)

Normalizing gradient datas...
Rearranging data to tensor for model classification...
Obtained regularized tensor.
Normalizing gradient datas...
Rearranging data to tensor for model classification...
Obtained regularized tensor.
Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1421
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1140
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1141
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1140
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1139
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1140
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1140
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1140
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1139
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1140
Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0907
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss

In [ ]:
# Run this section if you want to use CNN model

array1, headers1, indices1 = cnn.data_arranger(data1.df, resolution)
array2, headers2, indices2 = cnn.data_arranger(data2.df, resolution)

# The learning rate and epochs are proven to be working.

cnn_model = cnn.model_2D_create_compile(headers1, 0.05, resolution)

cnn_model, loss_hist = cnn.model_2D_train(cnn_model, array1, 3)
cnn_model, loss_hist = cnn.model_2D_train(cnn_model, array2, 3)

# Data Classification

Choose codes from these 3 codes below to run if you want to classify few individual data.

In [14]:
data3 = analyzer.Data("MORE_DATA/db_Y_0049.okc", resolution)

Trying to open the file at MORE_DATA/db_Y_0049.okc...
Obtained metadata of the file at MORE_DATA/db_Y_0049.okc.
This data is sliced.
Trying to open the file at MORE_DATA/db_Y_0049.okc...
Obtained metadata of the file at MORE_DATA/db_Y_0049.okc.
Obtain the time for this data.
Trying to open the file at MORE_DATA/db_Y_0049.okc...
Obtained metadata of the file at MORE_DATA/db_Y_0049.okc.
Finished mapping this data to your specified resolution.
Regularizing parameters...
Regularizing parameters...
Regularizing parameters...
Converting the temperature data to multi-dimensional array...
Calculating the gradient of temperature...
Finished calculating gradient.
Converting the velocity_magnitude data to multi-dimensional array...
Calculating the gradient of velocity_magnitude...
Finished calculating gradient.
Converting the z_velocity data to multi-dimensional array...
Calculating the gradient of z_velocity...
Finished calculating gradient.


In [15]:
# Run this code if your model choice is NN
array3, headers3, non_nan_indices3, num_grids3 = nn.data_arranger(data3.df)
data3.df = nn.model_classification(nn_model, array3, non_nan_indices3, num_grids3, data3.df, False)

Normalizing gradient datas...
Rearranging data to tensor for model classification...
Obtained regularized tensor.
1917/1917 ━━━━━━━━━━━━━━━━━━━━ 2s 786us/step
Finished classifying data.


In [ ]:
# Run this code if your model choice is CNN
array3, headers3, indices3 = cnn.data_arranger(data3.df, resolution)
data3.df = cnn.model_2D_classification(cnn_model, array3, indices3, data3.df, False)

Choose codes from these 3 codes below to run if you want to classify a series of data.

In [ ]:
list_paths_classify = [f"MORE_DATA/db_Y_{i:04d}.okc" for i in range(99)]

Data_classify = [analyzer.Data(path, list_paths_classify, resolution) for path in list_paths_classify] 

In [ ]:
# Run this code if your model choice is NN
for data in Data_classify:
    array, headers, non_nan_indices, num_grids = nn.data_arranger(data.df)
    data.df = nn.model_classification(nn_model, array, non_nan_indices, num_grids, data.df, False)

In [ ]:
# Run this code if your model choice is CNN
for data in Data_classify:
    array, headers, indices = cnn.data_arranger(data.df, data.resolution)
    data.df = cnn.model_2D_classification(cnn_model, array, indices, data.df, False)

# Data Export

Choose codes from these 2 codes below to run if you want to classify few individual data.

In [16]:
# Run this code to export the image
visualizer.plot_2D_df(data3.df, 'is_boundary', 'classification.png')

In [8]:
# Run this code to export the csv file
data3.df.to_csv('classification.csv', index=False)

Choose codes from these 2 codes below to run if you want to classify a series of data.

In [ ]:
# Run this code to export the image
for i, data in enumerate(Data_classify):
    visualizer.plot_2D_df(data.df, 'is_boundary', f'classification_{i}.png')

In [ ]:
# Run this code to export the csv file
for i, data in enumerate(Data_classify):
    data.df.to_csv(f'classification_{i}.csv', index=False)

# Temporary Code Zone
---

### Merge CSV

In [ ]:
import os
import csv

# 指定文件夹路径
path = '3D_DATA/small_db_result'
output_file = '3D_DATA/small_db.csv'

# 获取所有csv文件的文件名
files = [file for file in os.listdir(path) if file.endswith('.csv')]

# 打开输出文件，准备写入合并后的内容
with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    writer = None  # 创建一个CSV写入器，但先不初始化
    
    # 遍历所有CSV文件
    for index, filename in enumerate(files):
        # 显示进度
        print(f"merging file: {index + 1}/{len(files)}", flush=True)
        
        with open(os.path.join(path, filename), 'r', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            
            # 获取文件的第一行（标题）
            header = next(reader)
            
            # 如果是第一个文件，写入标题
            if writer is None:
                writer = csv.writer(outfile)
                writer.writerow(header)
            
            # 从第二行开始读取并写入
            for row in reader:
                writer.writerow(row)

print("All files merged successfully!")


### Plot Streamline

In [ ]:
from pyvista import examples

# Download the carotid mesh example
mesh = examples.download_carotid()
import pyvista as pv
from pyvista import examples

# Download the carotid mesh example
mesh = examples.download_carotid()

# Generate streamlines and source
streamlines, src = mesh.streamlines(
    return_source=True,
    max_time=100.0,
    initial_step_length=2.0,
    terminal_speed=0.1,
    n_points=25,
    source_radius=2.0,
    source_center=(133.1, 116.3, 5.0),
)

# Set up an off-screen plotter
p = pv.Plotter(off_screen=True)  # Enable offscreen rendering
p.add_mesh(mesh.outline(), color="k")
p.add_mesh(streamlines.tube(radius=0.15))
p.add_mesh(src)
p.add_mesh(mesh.contour([160]).extract_all_edges(), color="grey", opacity=0.25)
p.camera_position = [(182.0, 177.0, 50), (139, 105, 19), (-0.2, -0.2, 1)]

# Render and save the screenshot
p.export_html("carotid_streamlines.html")



### Test

In [ ]:
import pandas as pd
import numpy as np
import pyvista as pv
from scipy.interpolate import griddata



# Step 1: Read and preprocess your data
df = pd.read_csv('3D_DATA/ra10e7_result/ra10e7_100.csv')

# Keep only necessary columns
df = df[['x', 'z', 'x_velocity', 'z_velocity']].dropna()

# Get unique sorted x and z values
x_values = np.sort(df['x'].unique())
z_values = np.sort(df['z'].unique())

dx = np.diff(x_values).mean()

# Define grid points for interpolation
grid_x, grid_z = np.meshgrid(x_values, z_values, indexing='ij')

# Prepare the data for interpolation
points = df[['x', 'z']].values
u_values = df['x_velocity'].values
w_values = df['z_velocity'].values

# Interpolate velocities onto the grid
u_grid = griddata(points, u_values, (grid_x, grid_z), method='linear')
w_grid = griddata(points, w_values, (grid_x, grid_z), method='linear')

# Replace NaNs in interpolated data with zeros or nearest values
u_grid = np.nan_to_num(u_grid, nan=0.0)
w_grid = np.nan_to_num(w_grid, nan=0.0)

# Step 2: Create a structured grid
x_grid_3d = grid_x
y_grid_3d = np.zeros_like(grid_x)
z_grid_3d = grid_z
grid = pv.StructuredGrid(x_grid_3d, y_grid_3d, z_grid_3d)

# Flatten the arrays in Fortran order
u_flat = u_grid.ravel(order='F')
w_flat = w_grid.ravel(order='F')

# Create the velocity array
velocity = np.column_stack((u_flat, np.zeros_like(u_flat), w_flat))

# Assign the velocity data to the grid
grid.point_data['velocity'] = velocity

# Step 3: Compute streamlines
# Define seed points along the left edge
# Define a grid of seed points
x_seed, z_seed = np.meshgrid(
    np.linspace(x_values[0], x_values[-1], 4),
    np.linspace(z_values[0], z_values[-1], 4)
)
x_seed = x_seed.ravel()
z_seed = z_seed.ravel()
y_seed = np.zeros_like(x_seed)

# Combine into seed points
seed_points = np.column_stack((x_seed, y_seed, z_seed))
seed = pv.PolyData(seed_points)

# Compute streamlines from the seed points
streamlines = grid.streamlines_from_source(
    source=seed,
    vectors='velocity',
    integration_direction='both',
    max_time=100,
    initial_step_length=dx,
    terminal_speed=1e-5
)

# Step 4: Plot and export the streamlines
plotter = pv.Plotter(off_screen=True)
plotter.add_mesh(grid.outline(), color='k')
plotter.add_mesh(streamlines.tube(radius=dx*0.5), color='blue')
plotter.view_xy()

# Export to an HTML file
plotter.export_html('streamlines.html')
